In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score, log_loss, roc_curve, classification_report
)

# 1. Veri Setini Yükle ve Hazırla
data = load_breast_cancer()
X = data.data
y = data.target

# Veriyi Eğitim ve Test setlerine ayır (Veri setini görmeyen kısım test edilir)
# train_test_split fonksiyonu 4 değer döndürür: X_train, X_test, y_train, y_test.
# İlk satırda 5 değişken atanmaya çalışıldığı için 'ValueError' hatası alınmıştır.
# Hata düzeltmesi için aşağıdaki satır kullanılacaktır.
X_train, X_test, y_true, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# Burada 'y_true' aslında eğitim seti etiketlerini (y_train) temsil etmektedir.
# 'y_test' ise test seti etiketlerini temsil etmektedir.

# 2. Model Eğitimi (Logistic Regression)
# Model eğitimi için eğitim verileri (X_train) ve eğitim etiketleri (y_true) kullanılmalıdır.
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train, y_true)

# 3. Tahminler
# Tahmin Edilen Sınıflar (y_pred)
y_pred = model.predict(X_test)
# Tahmin Edilen Olasılıklar (y_proba) - ROC-AUC ve Log Loss için
y_proba = model.predict_proba(X_test)[:, 1] # Pozitif sınıf (Malign) olasılıkları

# Metrik hesaplamalarında kullanılacak gerçek etiketleri (y_true) test seti etiketleri (y_test) olarak belirliyoruz.
y_true = y_test

In [3]:
cm = confusion_matrix(y_true, y_pred)
TN, FP, FN, TP = cm.ravel()
print("## 5. Confusion Matrix\n")
print(f"sklearn çıktısı (TN, FP, FN, TP):\n{cm}\n")
print(f"   Gerçek Malign (1): {TP + FN} vaka")
print(f"   Gerçek Benign (0): {TN + FP} vaka")
print("-" * 30)

## 5. Confusion Matrix

sklearn çıktısı (TN, FP, FN, TP):
[[ 56   8]
 [  2 105]]

   Gerçek Malign (1): 107 vaka
   Gerçek Benign (0): 64 vaka
------------------------------


In [4]:
accuracy = accuracy_score(y_true, y_pred)
print("## 1. Accuracy (Doğruluk)\n")
print(f"Accuracy: {accuracy:.4f}")
print("-" * 30)

## 1. Accuracy (Doğruluk)

Accuracy: 0.9415
------------------------------


In [5]:
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("## 3. Precision, Recall ve F1-Score (Malign Sınıfı)\n")
print(f"Precision (Keskinlik): {precision:.4f}")
print(f"Recall (Duyarlılık): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("-" * 30)

# Tüm metrikleri gösteren Classification Report
print("\nClassification Report (Detaylı Sonuçlar):\n")
print(classification_report(y_true, y_pred, target_names=data.target_names))
print("-" * 30)

## 3. Precision, Recall ve F1-Score (Malign Sınıfı)

Precision (Keskinlik): 0.9292
Recall (Duyarlılık): 0.9813
F1-Score: 0.9545
------------------------------

Classification Report (Detaylı Sonuçlar):

              precision    recall  f1-score   support

   malignant       0.97      0.88      0.92        64
      benign       0.93      0.98      0.95       107

    accuracy                           0.94       171
   macro avg       0.95      0.93      0.94       171
weighted avg       0.94      0.94      0.94       171

------------------------------


In [6]:
auc_score = roc_auc_score(y_true, y_proba)
# ROC eğrisini çizmek için gerekli veriler
fpr, tpr, thresholds = roc_curve(y_true, y_proba)
#

print("## 4. ROC-AUC Skoru\n")
print(f"ROC-AUC Skoru: {auc_score:.4f}")
print("-" * 30)

## 4. ROC-AUC Skoru

ROC-AUC Skoru: 0.9873
------------------------------


In [7]:
logloss = log_loss(y_true, y_proba)
print("## 2. Loss (Kayıp Fonksiyonu)\n")
print(f"Log Loss: {logloss:.4f}")

## 2. Loss (Kayıp Fonksiyonu)

Log Loss: 0.1276


Önemli Yorum

Bu tıbbi teşhis senaryosunda, Recall (Duyarlılık) en kritik metrik olabilir.
 * FN (False Negative): Kanserli (Malign) bir hastayı sağlıklı (Benign) olarak yanlış sınıflandırmak, hayati tehlikeye yol açabilir. Bu hatanın maliyeti çok yüksektir.

 * Yüksek Recall değeri, modelin gerçek Malign vakalarının çoğunu doğru tespit ettiği anlamına gelir. Modelinizin Malign vakalarını kaçırmadığından emin olmak için bu metriği optimize etmelisiniz.